<a href="https://colab.research.google.com/github/baochi0212/Optimization/blob/master/TS_LNS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import some basic libraries 

In [ ]:
import random as rd
from copy import deepcopy
from math import sqrt
import numpy as np
import time

In [ ]:
import numpy as np
import json
import argparse

Overall idea: 
1. Create a list Pop with MaxPopSize initial solutions
- Each solution contains of M route of salesmen.
- Updated solution created by removing some nodes from list of nodes, then re-insert it in random route by function *insert* in order to find a better solution.
- Sorting Pop by worst cost of each solution with function *PopSorted*.
2. From Pop, keep (size(Pop)/f) best solutions from Pop
- f is a value that we decided. 
I.e: f = 2, MaxPopSize = 100 then keep 50 best solutions.
3. From each remaining solutions, perform it with LNS algorithm to find even better solution
- LNS: Perform (nofmutations) mutations by removing some random nodes from solutions then re-insert it in the solution again, if it's better update this solution as new solution.
- Two different ways to remove nodes: 

If the problem is m-ATSP (general case) then we use function *RmvR*, pick a random number of nodes from alp.N to beta.N.

If the problem is m-ATSP (direct case) then we use function *RmvP*, pick some seeds from N, then choose some nodes nearest to each seed and remove from solution, include seed itself, total number of remove nodes is random from alp.sqrt(N) to beta.sqrt(N), since it's more targerted.

alp and beta are values we decide.
4. We start with an initial number of mutations which we decide, then after each iteration, while the size of Pop decrease, we also increase the number of mutations.
5. When the size of Pop go below MinPopSize, return the best solution from remaning solutions in Pop.

In [ ]:
#return the worst cost of a solution
def worstCost(cr,c):                  
    wc = -1
    for path in cr:
        cost = 0
        for i in range(len(path)-1):
            cost += c[path[i]][path[i+1]]
        if cost > wc:
            wc = cost
    return wc

#the cost of a route 
def cost(route, c):
    cost = 0
    for i in range(len(route)-1):
        cost += c[route[i]][route[i+1]]
    return cost

Algorithm 1: Remove a node from route then re-insert again.

In [ ]:
#algorithm 1: NodeInsert
def rmvNodeRandom(nodes):             
    rmvPos = rd.randint(0, len(nodes)-1)
    rmvNode = nodes.pop(rmvPos)
    return rmvNode

def addNode(route, i, n_j):             
    r_ = deepcopy(route)
    r_.insert(i+1, n_j)
    return r_

def insert(sol, nodes, c):    #change the original solution
    while len(nodes) != 0:
        minwcost = 1e20   
        n_j = rmvNodeRandom(nodes)                   
        for r in sol:                                  
            length = len(r)-1
            for i in range(length):
                rP = addNode(r, i, n_j)
                cost_rP = cost(rP, c)
                if cost_rP < minwcost:
                    bestr, bestrP = deepcopy(r), deepcopy(rP)
                    minwcost = cost_rP
        sol.remove(bestr)
        sol.append(bestrP)
    return sol
#end algorithm 1

Algorithm 2: Remove k random nodes from a solution.
- RmvR: Use for general case
- RmvP: Use for direct case

In [ ]:
#algorithm 2: NodeRemoval - 2 strategy (m-ATSP, m-TSP)
def pickRandom(N, nofnodes):
    return rd.sample(N, nofnodes)

def rmvNode(cur, n):
    for i in cur:
        if n in i:
            i.remove(n)
            break
    return cur

def nearestNode(s, nofnodesP, c):
    s_ = []
    for i in range(1, len(c[s])):
        s_.append([c[s][i], i])
    sorted(s_)
    sn = []
    for i in range(nofnodesP):
        sn.append(s_[i][1])
    return sn    

#first method use for m-ATSP
def RmvR(sol, nofnodes, N):       #change the original solution
    free = pickRandom(N, nofnodes) 
    for n in free:
        rmvNode(sol,n)
    return sol, free
#end method

#second method use for m-TSP
def RmvP(sol, nofnodes, nofseeds, N, c):  #change the original solution 
    nofnodesP = int(nofnodes/nofseeds)
    seeds = pickRandom(N, nofseeds) 
    free = []

    for s in seeds:
        nn = nearestNode(s, nofnodesP, c) #include s itself
        for n in nn:
            rmvNode(sol, n)
            free.append(n)

    return sol, free
#end method

#end algorithm 2

Algorithm 3: Large neighbor search - mutating some random nodes using algo 2 then re-insert it with algo 1.


In [ ]:
#algorithm 3: LNS
def fitness(sol, c):
    return 1/worstCost(sol, c)

def LNS(sol, nofmutations, rmvopt, alp, beta, N, c, n):
    if rmvopt == "rand":
        lower, upper = int(alp*n), int(beta*n)
    elif rmvopt == "proximity":
        lower, upper = int(alp*sqrt(n)), int(beta*sqrt(n))
    
    wc = worstCost(sol, c)

    for i in range(nofmutations):
        rmv = rd.randint(lower, upper)
        if rmvopt == "rand":
            tmp, free = RmvR(deepcopy(sol), rmv, N)
        elif rmvopt == "proximity":
            seeds = rd.randint(1, int(upper/10))
            tmp, free = RmvP(deepcopy(sol), rmv, seeds, N, c)
    
        new = insert(tmp, free, c)
        newwc = worstCost(new, c)
        if newwc < wc:
            wc = newwc
            sol = deepcopy(new)
    return sol
#end function

Algorithm 4: Main algorithm 
- Build MaxPopSize initial soutions, then select nofsol/f fittest solutions to keep with f is a given value.
- Each solution in remaining solutions goes through algo 3 to find even better result.
- When only MinPopSize solutions left, pick the fittest solution. 

In [ ]:
#algorithm 4: TS-LNS - Main algorithm
def initLNSMutations():           #simple version
    return 200

def adjustLNSMutations(mut):      #simple version
    return mut + 200

def getFittest(Pop, nofsol_kept): 
    return Pop[:nofsol_kept]

def PopSorted(Pop, c):
    worstlist = []
    for i in range(len(Pop)):
        worstlist.append([worstCost(Pop[i], c), Pop[i]])
    worstlist.sort()
    for i in range(len(worstlist)):
        worstlist[i] = worstlist[i][1]
    return worstlist

def TS_LNS(N, M, rmvopt, node0, MaxPopSize, MinPopSize, f, alp, beta, c, k):
    nullsol = []
    Pop = list()
    
    for i in range(M):
        nullsol += [[node0, node0]]
    start = time.time()
    for i in range(MaxPopSize):
        n = deepcopy(N)
        rd.shuffle(n)
        nsol = deepcopy(nullsol)
        Pop.append(insert(nsol, n, c))

    Pop = PopSorted(Pop, c)               
    mut = initLNSMutations()    
    end = time.time()
    print(end-start)
    while len(Pop) > MinPopSize:
        Pop = getFittest(Pop, int(len(Pop)/f))         
        
        #############
        for sol in Pop:
            sol = LNS(sol, mut, rmvopt, alp, beta, N, c, k)

        PopSorted(Pop, c)             
        mut = adjustLNSMutations(mut)              

    return getFittest(Pop, 1)[0]
#end function

Random datasets generator


In [ ]:
def gena(N0, K0):
    d = [0]
    for i in range(N0):
        d.append(rd.randint(1, 100))
    
    t = []
    for i in range(N0+1):
        row = []
        for j in range(N0+1):
            if i == j:
                row.append(0)
            else:
                row.append(rd.randint(1,100))
        t.append(row)
    c = np.array(t) + np.array(d)
    c = c.tolist()
    
    for i in range(len(c)):
        c[i][i] = 0

    return t, d, c
def gens(N0, K0):
  c = [[0 for i in range(N0+1)] for j in range(N0+1)]
  for i in range(N0+1):
    for j in range(N0+1):
      if i != j:
        point = rd.randint(1, 100)
        c[i][j], c[j][i] = point, point
  return c
    

Datasets lib

In [ ]:
def datasets(filename):
    with open(filename) as f:
        N, M = [int(i) for i in f.readline().split()]
        d = [0] + [int(i) for i in f.readline().split()]
        t = []
        for i in range(N+1):
            arr = [int(i) for i in f.readline().split()]
            t.append(arr)
        c = np.array(t) + np.array(d)
        c = c.tolist()
        for i in range(len(c)):
            c[i][i] = 0
        return N, M, d, t, c

In [ ]:
if __name__ == "__main__":
  case = input("mATSP / mTSP (1/2): ")
  if case == "1":
    n, M = int(input("Number of customers: ")), int(input("Number of salesmen: "))
    t, d, c = gena(n, M)
    rmvopt = "rand"
    alp = .2
    beta = .4
    # print("Time table for going from point to point: ")
    # print(np.array(t))
    # print("Time to fix something for customers: ")
    # print(np.array(d))
  else:
    # filename = input("Choose a datasets file: ")
    # N, M, d, t, c = datasets(filename)
    n, M = int(input("Number of customers: ")), int(input("Number of salesmen: "))
    c = gens(n, M)
    rmvopt = "proximity"
    alp = 1
    beta = 4
    # print(np.array(c))

mATSP / mTSP (1/2): 1
Number of customers: 30
Number of salesmen: 3


In [ ]:
  N = []
  for i in range(n):
    N.append(i+1)
  node0 = 0
  MaxPopSize = 100
  MinPopSize = 6    
  f = 2
  sol = TS_LNS(N, M, rmvopt, node0, MaxPopSize, MinPopSize, f, alp, beta, c, n)
  for i in sol:
    print(i)
  print(worstCost(sol, c))

0.7543118000030518
[0, 6, 24, 25, 9, 1, 30, 26, 5, 12, 0]
[0, 8, 17, 4, 21, 20, 7, 19, 11, 16, 15, 22, 13, 0]
[0, 18, 23, 28, 2, 3, 14, 10, 29, 27, 0]
733


In [ ]:
# parser = argparse.ArgumentParser("INPUT")
# parser.add_argument('--input', type=str, default='sample0.json')
# if __name__ == '__main__':
# 	args = parser.parse_args()
# 	name = args.input
with open('sample1.json', 'r') as f:
	input = json.load(f)
n, M, d, t = input['N'], input['k'], input['d'], input['t']
N = []
for i in range(n):
    N.append(i+1)
c = np.array(t) + np.array(d)
c = c.tolist()
node0 = 0
MaxPopSize = 100
MinPopSize = 6
f = 2
alp = .1
beta = .3
rmvopt = "rand"

start = time.time()
sol = TS_LNS(N, M, rmvopt, node0, MaxPopSize, MinPopSize, f, alp, beta, c, n)
end = time.time()
print(sol)
print(worstCost(sol, c))
print(end-start)

8.44395899772644
[[0, 5, 82, 27, 59, 30, 41, 40, 11, 64, 88, 42, 0], [0, 22, 13, 65, 38, 71, 2, 47, 3, 87, 24, 0], [0, 97, 75, 43, 93, 57, 26, 83, 55, 20, 46, 0], [0, 85, 63, 48, 62, 18, 94, 51, 12, 29, 66, 56, 67, 0], [0, 32, 80, 91, 76, 33, 37, 15, 89, 49, 44, 34, 0], [0, 28, 23, 31, 86, 54, 95, 21, 1, 19, 50, 74, 0], [0, 92, 36, 25, 7, 53, 61, 39, 58, 99, 81, 8, 0], [0, 73, 90, 77, 69, 10, 35, 16, 70, 4, 14, 68, 96, 0], [0, 60, 84, 79, 52, 45, 17, 98, 6, 78, 72, 9, 0]]
35
832.7651796340942
